<a href="https://colab.research.google.com/github/E-wave112/ml_proj1/blob/master/spooknlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#spooky nlp
##load datsets
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
train_df = pd.read_csv('/content/drive/MyDrive/spook/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/spook/test.csv')
submission_file = pd.read_csv('/content/drive/MyDrive/spook/sample_submission.csv')
id = test_df['id']
test_df.drop(test_df[['id']],1,inplace=True)
train_df.drop(train_df[['id']],1,inplace=True)


label_enc = LabelEncoder()

train_df['author'].replace('EAP',1,inplace=True)
train_df['author'].replace('HPL',2,inplace=True)
train_df['author'].replace('MWS',3,inplace=True)

print(train_df.shape)

##tokeize our text

count_vectorizer = CountVectorizer()

train_df['text'] = count_vectorizer.fit_transform(train_df['text']).todense()
test_df['text'] = count_vectorizer.transform(test_df['text']).todense()
print(train_df[:10])

log_clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')
train_df = np.array(train_df).reshape(len(train_df),-1)
train_df = pd.DataFrame(train_df,columns=['text','author'])
print(type(train_df))
# train_df['author'] = np.array(train_df['author']).reshape(len(train_df['author']),-1)
log_clf.fit(train_df[['text']],train_df[['author']])

test_df = np.array(test_df).reshape(len(test_df),-1)
pred = log_clf.predict_proba(test_df)
submit = pd.DataFrame({
    'id':id,
    'EAP':pred[:,0],
    'HPL':pred[:,1],
    'MWS':pred[:,2]
})
submit.to_csv('/content/drive/MyDrive/spook/submission2.csv',index=False)

(19579, 2)
   text  author
0     0       1
1     0       2
2     0       1
3     0       3
4     0       2
5     0       3
6     0       1
7     0       1
8     0       1
9     0       3
<class 'pandas.core.frame.DataFrame'>


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
###implementing dl and lstm on this model
import re
import pandas as pd
from pandas import DataFrame
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('dark_background')
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D


train_df = pd.read_csv('/content/drive/MyDrive/spook/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/spook/test.csv')
submission_file = pd.read_csv('/content/drive/MyDrive/spook/sample_submission.csv')

id = test_df['id']
test_df.drop(test_df[['id']],1,inplace=True)
train_df.drop(train_df[['id']],1,inplace=True)




train_df['author'].replace('EAP',1,inplace=True)
train_df['author'].replace('HPL',2,inplace=True)
train_df['author'].replace('MWS',3,inplace=True)

def remove_nums(num):
    return ''.join(i for i in num if not i.isdigit())

train_df['text'] = train_df['text'].apply(remove_nums)
test_df['text'] = test_df['text'].apply(remove_nums)


def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words
    
# train_df['text'] = train_df['text'].apply(remove_stopwords)
# test_df['text'] = test_df['text'].apply(remove_stopwords)
# print(type(train_df['text']))
replace_list = {r"i'm": 'i am',
                r"'re": ' are',
                r"let’s": 'let us',
                r"'s":  ' is',
                r"'ve": ' have',
                r"can't": 'can not',
                r"cannot": 'can not',
                r"shan’t": 'shall not',
                r"n't": ' not',
                r"'d": ' would',
                r"'ll": ' will',
                r"'scuse": 'excuse',
                ',': ' ,',
                '.': ' .',
                '!': ' !',
                '?': ' ?',
                '\s+': ' '}
# def clean_text(text):
#     text = text.lower()
#     for s in replace_list:
#         text = text.replace(s, replace_list[s])
#     text = ' '.join(text.split())
#     return text



# train_df['text'] = train_df['text'].apply(lambda x:clean_text(x))
# test_df['text'] = test_df['text'].apply(lambda x:clean_text(x))


max_words = 8192
tokenizer = Tokenizer(
    num_words = max_words,
    filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~',
    lower=True
)

# phrase_len = train_df['text'].apply(lambda p:len(p.split(' ')))


# max_phrase_len = phrase_len.max()
# print(max_phrase_len)
tokenizer.fit_on_texts(train_df['text'])
tokenizer.fit_on_texts(test_df['text'])

X = tokenizer.texts_to_sequences(train_df['text'])
X_test = tokenizer.texts_to_sequences(test_df['text'])


vocab_size = len(tokenizer.word_index) + 1

#print(vocab_size)
X  = pad_sequences(X,maxlen=100)
X_test = pad_sequences(X_test,maxlen=100)
Y = np.array(train_df['author'])
Y =to_categorical(Y)
Y = Y[:,[1,2,3]]
print(Y[:5])

batch_size = 512
epochs = 8

model_lstm = Sequential()
model_lstm.add(Embedding(vocab_size,100,input_length=100))
#tf.keras.layers.Flatten()
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3,activation='relu'))
model_lstm.add(Dense(3, activation='softmax'))
model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['AUC']
)

history = model_lstm.fit(
    X,
    Y,
    validation_split = 0.15,
    epochs = epochs,
    batch_size = 256,callbacks=EarlyStopping(monitor="val_loss",
                                              min_delta=0.001,
                                              patience=5)
)


pred = model_lstm.predict_proba(X_test)
print(pred[:5])

submit = pd.DataFrame({
    'id':id,
    'EAP':pred[:,0],
    'HPL':pred[:,1],
    'MWS':pred[:,2]
})
submit.to_csv('/content/drive/MyDrive/spook/submissionfirst.csv',index=False)

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Epoch 1/8
66/66 [==============================] - 165s 2s/step - loss: 184.2829 - auc: 0.5803 - val_loss: 1.0567 - val_auc: 0.6741
Epoch 2/8
66/66 [==============================] - 159s 2s/step - loss: 6.2118 - auc: 0.6345 - val_loss: 1.0423 - val_auc: 0.6781
Epoch 3/8
66/66 [==============================] - 158s 2s/step - loss: 86.0316 - auc: 0.6649 - val_loss: 1.0211 - val_auc: 0.6950
Epoch 4/8
66/66 [==============================] - 159s 2s/step - loss: 1.0060 - auc: 0.7124 - val_loss: 0.9973 - val_auc: 0.7224
Epoch 5/8
66/66 [==============================] - 158s 2s/step - loss: 0.9717 - auc: 0.7618 - val_loss: 0.9695 - val_auc: 0.7501
Epoch 6/8
66/66 [==============================] - 159s 2s/step - loss: 0.9460 - auc: 0.7717 - val_loss: 1.3326 - val_auc: 0.6818
Epoch 7/8
66/66 [==============================] - 161s 2s/step - loss: 1.3100 - auc: 0.6372 - val_loss: 0.9956 - val_auc: 0.7302
Epoch 8/8
66/66 [=========

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


[[0.43521097 0.27547234 0.2893167 ]
 [0.5939909  0.20157175 0.20443732]
 [0.48781687 0.2682728  0.2439103 ]
 [0.56647307 0.2393548  0.19417214]
 [0.45242772 0.26294956 0.28462276]]


In [ ]:
import re
import pandas as pd
from pandas import DataFrame
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('dark_background')
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.linear_model import LogisticRegression


##data loading
train_df = pd.read_csv('/content/drive/MyDrive/spook/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/spook/test.csv')
submission_file = pd.read_csv('/content/drive/MyDrive/spook/sample_submission.csv')

id = test_df['id']
test_df.drop(test_df[['id']],1,inplace=True)
train_df.drop(train_df[['id']],1,inplace=True)




train_df['author'].replace('EAP',1,inplace=True)
train_df['author'].replace('HPL',2,inplace=True)
train_df['author'].replace('MWS',3,inplace=True)

def remove_nums(num):
    return ''.join(i for i in num if not i.isdigit())

train_df['text'] = train_df['text'].apply(remove_nums)
test_df['text'] = test_df['text'].apply(remove_nums)


def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words
    
train_df['text'] = train_df['text'].apply(remove_stopwords)
test_df['text'] = test_df['text'].apply(remove_stopwords)

max_words = 10000
tokenizer = Tokenizer(
    num_words = max_words,
    filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~',
    lower=True
)

##vectorize characters into numbers
tokenizer.fit_on_texts(train_df['text'])
tokenizer.fit_on_texts(test_df['text'])

X = tokenizer.texts_to_sequences(train_df['text'])
X_test = tokenizer.texts_to_sequences(test_df['text'])

X  = pad_sequences(X,maxlen=100)
X_test = pad_sequences(X_test,maxlen=100)
X = X.flatten()
X_test = X_test.flatten()
X = X.reshape(19579,-1)
X_test = X_test.reshape(len(X_test),-1)
Y = np.array(train_df['author'])
Y = Y.reshape(len(Y),-1)
print(Y[:4])


log_clf = LogisticRegression(multi_class='multinomial',C=0.1,penalty='l2')

log_clf.fit(X,Y)

pred = log_clf.predict_proba(Y)

submit = pd.DataFrame({
    'id':id,
    'EAP':pred[:,0],
    'HPL':pred[:,1],
    'MWS':pred[:,2]
})

submit.to_csv('/content/drive/MyDrive/spook/submissionlogit.csv',index=False)

[[1]
 [2]
 [1]
 [3]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored

In [ ]:
a = np.array([[1,2,3,4,5],[1,2,3,4,5]])
a = a.reshape(4,-1)
print(a)

ValueError: ignored

In [ ]:
###implementing dl and lstm on this model
import nltk
nltk.download('stopwords')
from keras.preprocessing.sequence import pad_sequences
import numpy as np

a = np.array([[1],[2,3,4],[5,7,8]])
a = pad_sequences(a,padding='post',maxlen=2)
print(a)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[[1 0]
 [3 4]
 [5 7]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
